In [1]:
import numpy as np
import os
from ECG.eager_ops import expected_gradients as eg
import tensorflow as tf

In [2]:
path = "./eval_data_10k/cnn_ph_eg/"

In [3]:
os.listdir(path)

['cms_cv1.npy',
 'cms_cv10.npy',
 'cms_cv2.npy',
 'cms_cv3.npy',
 'cms_cv4.npy',
 'cms_cv5.npy',
 'cms_cv6.npy',
 'cms_cv7.npy',
 'cms_cv8.npy',
 'cms_cv9.npy',
 'history_cv1.npy',
 'history_cv10.npy',
 'history_cv2.npy',
 'history_cv3.npy',
 'history_cv4.npy',
 'history_cv5.npy',
 'history_cv6.npy',
 'history_cv7.npy',
 'history_cv8.npy',
 'history_cv9.npy',
 'Model_cv1.h5',
 'Model_cv10.h5',
 'Model_cv2.h5',
 'Model_cv3.h5',
 'Model_cv4.h5',
 'Model_cv5.h5',
 'Model_cv6.h5',
 'Model_cv7.h5',
 'Model_cv8.h5',
 'Model_cv9.h5',
 'preds_cv1.npy',
 'preds_cv10.npy',
 'preds_cv2.npy',
 'preds_cv3.npy',
 'preds_cv4.npy',
 'preds_cv5.npy',
 'preds_cv6.npy',
 'preds_cv7.npy',
 'preds_cv8.npy',
 'preds_cv9.npy',
 'results_cv1.npy',
 'results_cv10.npy',
 'results_cv2.npy',
 'results_cv3.npy',
 'results_cv4.npy',
 'results_cv5.npy',
 'results_cv6.npy',
 'results_cv7.npy',
 'results_cv8.npy',
 'results_cv9.npy']

In [4]:
#cv1
m_cv1 = tf.keras.models.load_model(path+"Model_cv1.h5")
preds = np.load(path+"preds_cv1.npy")

In [12]:
test_y = np.load("./eval_data_10k/teset_y.npy", allow_pickle=True)
test_x = np.load("./eval_data_10k/teset_x.npy", allow_pickle=True)

In [31]:
att = eg(inputs=test_x[0].reshape(-1, test_x[0].shape[1], 1).astype(np.float32), labels=tf.keras.utils.to_categorical(test_y[0].astype(np.float32)), model=m_cv1)

In [ ]:
tf.keras.utils.to_categorical(test_y[0]).shape

(10888, 9)

In [29]:
test_x.reshape(-1, test_x.shape[1], 1).astype(np.float32)

IndexError: tuple index out of range